In [8]:
!pip install openpyxlf
!pip install optuna
!pip install catboost
!pip install category-encoders

ERROR: Could not find a version that satisfies the requirement openpyxlf (from versions: none)
ERROR: No matching distribution found for openpyxlf

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: C:\Users\bychk\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: C:\Users\bychk\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: C:\Users\bychk\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: C:\Users\bychk\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [9]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from category_encoders.target_encoder import TargetEncoder
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import optuna

In [10]:
train = pd.read_csv("final_train.csv")
test = pd.read_csv("final_test.csv")

### Ridge регрессия с L2 регуляризацией

In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3980 entries, 0 to 3979
Data columns (total 47 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   предмет           3980 non-null   object 
 1   ФИО               3980 non-null   object 
 2   tutor_head_tags   3980 non-null   object 
 3   tutor_rating      3980 non-null   float64
 4   tutor_reviews     3980 non-null   int64  
 5   description       3980 non-null   object 
 6   experience_desc   3980 non-null   object 
 7   Education_1       3980 non-null   object 
 8   Desc_Education_1  3980 non-null   object 
 9   Education_2       3980 non-null   object 
 10  Desc_Education_2  3980 non-null   object 
 11  status            3980 non-null   object 
 12  experience        3980 non-null   int64  
 13  categories        3980 non-null   object 
 14  mean_price        3980 non-null   float64
 15  school            3980 non-null   int64  
 16  ege               3980 non-null   int64  


In [12]:
ridge_feat = ['tutor_rating', 'tutor_reviews', 'experience', 'school', 'ege', 'oge', 'geom', 'matan', 'linal',
       'teorver'] + [f'e_{i}' for i in range(24)] 
X_train = train[ridge_feat]
y_train = train['mean_price']
ridge_grid = {'max_iter': [50, 250, 750], 'alpha': [0.02, 2, 20]}
gc_regressor = GridSearchCV(param_grid=ridge_grid,
                            estimator=Ridge())
gc_regressor.fit(X_train, y_train)

GridSearchCV(estimator=Ridge(),
             param_grid={'alpha': [0.02, 2, 20], 'max_iter': [50, 250, 750]})

In [13]:
train[ridge_feat].isna().sum()

tutor_rating     0
tutor_reviews    0
experience       0
school           0
ege              0
oge              0
geom             0
matan            0
linal            0
teorver          0
e_0              0
e_1              0
e_2              0
e_3              0
e_4              0
e_5              0
e_6              0
e_7              0
e_8              0
e_9              0
e_10             0
e_11             0
e_12             0
e_13             0
e_14             0
e_15             0
e_16             0
e_17             0
e_18             0
e_19             0
e_20             0
e_21             0
e_22             0
e_23             0
dtype: int64

In [14]:
sample_submit = pd.read_csv("sample_submit.csv")

In [15]:
sample_submit['mean_price'] =gc_regressor.predict(test[ridge_feat])
sample_submit

,index,mean_price
0,0,17.345290
1,1,14.307629
2,2,17.779207
3,3,12.675455
4,4,11.931628
...,...,...
1511,1511,15.131097
1512,1512,18.409434
1513,1513,13.929438
1514,1514,15.208841


In [16]:
sample_submit.to_csv('ridge_1st.csv', index=None)

### Случайный лес

In [17]:
cat_features = [c for c in test.columns if c not in ridge_feat + ['categories', 'tutor_head_tags']]
X_train = train[ridge_feat + cat_features]
y_train = train['mean_price']
encoder = TargetEncoder(cols=cat_features)
X_train = encoder.fit_transform(X_train, y_train)
forest_grid = {'n_estimators': [250, 500], 'max_depth': [2, 3, 4, 6]}
gc_regressor = GridSearchCV(param_grid=forest_grid,
                            estimator=RandomForestRegressor())
gc_regressor.fit(X_train, y_train)

GridSearchCV(estimator=RandomForestRegressor(),
             param_grid={'max_depth': [2, 3, 4, 6], 'n_estimators': [250, 500]})

In [18]:
forest_test = encoder.transform(test[ridge_feat + cat_features])
sample_submit['mean_price'] = gc_regressor.predict(forest_test)
sample_submit

,index,mean_price
0,0,14.150070
1,1,14.150070
2,2,14.150070
3,3,14.150070
4,4,7.522546
...,...,...
1511,1511,14.150070
1512,1512,14.150070
1513,1513,14.150070
1514,1514,14.150070


In [19]:
sample_submit.to_csv('forest_1st.csv', index=None)

In [20]:
train.head()

,предмет,ФИО,tutor_head_tags,tutor_rating,tutor_reviews,description,experience_desc,Education_1,Desc_Education_1,Education_2,...,e_14,e_15,e_16,e_17,e_18,e_19,e_20,e_21,e_22,e_23
0,matematika,Репетитор по математике Назарова Анастасия Ан...,"['Школьная программа', 'Подготовка к ЕГЭ', 'По...",4.0,7,Репетитор не предоставил о себе дополн...,неизвестно,Вологодский государственный педагогический ун...,"Год окончания: 2017, Факультет: факультет при...",Череповецкий государственный университет,...,0.022926,-0.033269,0.035080,-0.028139,-0.013984,-0.012765,-0.055424,-0.015273,-0.068392,-0.115279
1,informatika,Репетитор по информатике Мотроненко Инна Конст...,"['Школьная программа', 'Подготовка к ЕГЭ', 'По...",0.0,0,"Выезд за город: Балашиха-1, Балашиха-2...",Московская Школа (учитель ...,Московский городской педагогический университет,"Год окончания: 2008, Факультет: Математически...",Московский городской педагогический университ...,...,0.093290,0.003983,-0.029426,-0.008604,-0.116641,-0.048152,0.112442,-0.125820,0.133195,-0.121066
2,matematika,Репетитор по математике Логинова Елизавета Ал...,"['Школьная программа', 'Математический анализ'...",4.0,19,"Здравствуйте! Меня зовут Елизавета, о...",неизвестно,Санкт-Петербургский государственный университет,"Год окончания: 2021, Факультет: физический фа...",неизвестно,...,0.003198,0.040983,-0.015660,-0.000587,-0.006566,-0.008209,0.005348,0.039918,-0.000853,-0.007152
3,matematika,Репетитор по математике Харланов Олег Викторо...,"['Школьная программа', 'Подготовка к ЕГЭ', 'По...",0.0,0,"Целеустремленный, ориентирован на резу...",преподавателем ...,Приазовский государственный технический униве...,"Год окончания: 2005, Факультет: Энергетически...",неизвестно,...,0.004849,0.045662,0.040038,0.025386,0.006353,-0.014001,-0.018947,-0.002155,0.003224,0.028982
4,matematika,Репетитор по математике Позябин Дмитрий Евген...,"['Школьная программа', 'Подготовка к ЕГЭ', 'По...",0.0,0,Репетитор не предоставил о себе дополн...,неизвестно,Московский государственный технический универ...,"Год окончания: 2019, Факультет: факультет эне...",неизвестно,...,0.043994,-0.015761,-0.006996,-0.048674,0.014612,-0.008134,-0.026880,-0.062674,0.013221,0.009322


### Градиентный бустинг

In [21]:
cat_features = [c for c in test.columns if c not in ridge_feat + ['categories', 'tutor_head_tags']]
X_train = train[ridge_feat + cat_features]
y_train = train['mean_price']

In [22]:
def objective(trial):
    train_x, valid_x, train_y, valid_y = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

    param = {
        "learning_rate": trial.suggest_float("learning_rate", 0.003, 0.3),
        "max_depth": trial.suggest_int("max_depth", 2, 6),
    }
    cat = CatBoostRegressor(cat_features=cat_features, **param)

    cat.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=False, early_stopping_rounds=20)

    preds = cat.predict(valid_x)
    rmse = mean_squared_error(valid_y, preds, squared=False)
    return rmse

In [23]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=17)
trial = study.best_trial

[I 2023-07-30 22:38:18,353] A new study created in memory with name: no-name-8bfdf3cd-b617-4be3-92fd-4add0e94a266
[I 2023-07-30 22:38:20,212] Trial 0 finished with value: 7.416042895085206 and parameters: {'learning_rate': 0.18162655445204506, 'max_depth': 2}. Best is trial 0 with value: 7.416042895085206.
[I 2023-07-30 22:38:23,935] Trial 1 finished with value: 7.463645610435271 and parameters: {'learning_rate': 0.06832230505016743, 'max_depth': 2}. Best is trial 0 with value: 7.416042895085206.
[I 2023-07-30 22:38:26,393] Trial 2 finished with value: 7.437430167271866 and parameters: {'learning_rate': 0.18038865108718183, 'max_depth': 3}. Best is trial 0 with value: 7.416042895085206.
[I 2023-07-30 22:38:40,572] Trial 3 finished with value: 7.4039203712861745 and parameters: {'learning_rate': 0.035814576325527876, 'max_depth': 5}. Best is trial 3 with value: 7.4039203712861745.
[I 2023-07-30 22:38:44,214] Trial 4 finished with value: 7.428379121690988 and parameters: {'learning_rate'

In [24]:
best_cat = CatBoostRegressor(cat_features=cat_features, **study.best_trial.params)
train_x, valid_x, train_y, valid_y = train_test_split(X_train, y_train, test_size=0.25, random_state=0)
best_cat.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=0, early_stopping_rounds=20)
n_estimators = best_cat.best_iteration_
final_cat = CatBoostRegressor(cat_features=cat_features, iterations=n_estimators, **trial.params)
final_cat.fit(X_train, y_train)

0:	learn: 8.2501777	total: 28.2ms	remaining: 6.52s
1:	learn: 8.1957683	total: 93.1ms	remaining: 10.7s
2:	learn: 8.1459736	total: 143ms	remaining: 10.9s
3:	learn: 8.0950522	total: 182ms	remaining: 10.4s
4:	learn: 8.0448457	total: 229ms	remaining: 10.4s
5:	learn: 8.0095042	total: 284ms	remaining: 10.7s
6:	learn: 7.9687014	total: 342ms	remaining: 11s
7:	learn: 7.9278174	total: 386ms	remaining: 10.8s
8:	learn: 7.8966077	total: 421ms	remaining: 10.4s
9:	learn: 7.8620188	total: 473ms	remaining: 10.5s
10:	learn: 7.8380269	total: 510ms	remaining: 10.3s
11:	learn: 7.8037003	total: 551ms	remaining: 10.1s
12:	learn: 7.7742132	total: 589ms	remaining: 9.93s
13:	learn: 7.7477722	total: 662ms	remaining: 10.3s
14:	learn: 7.7224475	total: 718ms	remaining: 10.4s
15:	learn: 7.6967620	total: 779ms	remaining: 10.5s
16:	learn: 7.6698110	total: 815ms	remaining: 10.3s
17:	learn: 7.6398818	total: 872ms	remaining: 10.4s
18:	learn: 7.6105344	total: 904ms	remaining: 10.1s
19:	learn: 7.5838186	total: 957ms	remaini

In [25]:
sample_submit['mean_price'] = final_cat.predict(test[ridge_feat + cat_features])
sample_submit

,index,mean_price
0,0,16.871157
1,1,13.275435
2,2,19.003542
3,3,12.361694
4,4,7.757493
...,...,...
1511,1511,13.116402
1512,1512,17.673263
1513,1513,15.946368
1514,1514,13.909862


In [26]:
sample_submit.to_csv('boosting_1st.csv', index=None)